# se.plan for large areas and batch processing

Occasionally it is needed to run a se.plan scenario for an area that is too large to be computed on the fly or when you have many scenarios you would like to run quickly without using the GUI. This notebook seeks to help facilitate those use cases. 

In this notebook I'll demonstrate how to use a previously saved se.plan recipe to run se.plan on a new area and save dashboard results. 

Specifically we will:

1. Load a previous se.plan recipe
2. Add a new AOI that would be larger than typically usable in the se.plan GUI
3. Calculate the dashboard statistics for the new aoi and 4 new sub aois
4. Run se.plan
5. Get dashboard statistics
6. Export the new recipe to a GEE asset and export our statistics to Google Drive as a CSV



### Step 1 - Load a previous se.plan recipe

We use a previous se.plan recipe to supply the answers from the questionnaire. se.plan uses a specific formatting schema when creating the recipes which is not the most fun to edit or interact with. If you are planning on running a few different scenarios for multiple regions I would suggest using the se.plan GUI to save the recipes for each plan then navigate back here to apply them programmatically.

In [ ]:
# import libaraies needed  and initialize to GEE.
from component import scripts
import ee
import json
import geemap
ee.Initialize()

In [ ]:


path = '/home/jdilger/module_results/se.plan/SLV/recipe_2022-05-23.json'
with open(path, 'r') as f:
  recipe = json.load(f)

# get priorities and constraints AS STRINGS
# wlc function expects this
priorities = recipe['question_model']['priorities']
constraints = recipe['question_model']['constraints']
layer_list = recipe['layer_model']['layer_list']

print('wlc priorities input type should be a string :',type(priorities))
print('wlc constraints input type should be a string :',type(constraints))
print('layer_list should be a list :',type(layer_list))

### Step 2 - Add a new AOI

Although each recipe has an aoi associated with it we will instead use a new area that we define. The way the aoi is saved in the recipe is not as straight forward since you can choose from preloaded collections uploaded shapefiles or personal GEE assets. For this notebook we will use a prepared aoi that encompasses Vietnam, Cambodia, Laos, and Thailand (aoi). 

Additionally, we will use those country borders as sub aois for generating dashboard results from the se.plan restoration suitability analysis.


In [ ]:
aoi = ee.FeatureCollection('users/jdilger/mekong-test')

fao = ee.FeatureCollection("FAO/GAUL/2015/level0")
points = ee.Geometry.MultiPoint(
        [[104.75271920718572, 12.738189346725711],
         [101.80838326968572, 16.184391550716192],
         [108.31228951968572, 13.850129627946572],
         [106.15896920718572, 16.184391550716192]])

sub_aois = fao.filterBounds(points)

### Step 3 - run se.plan

With our new aoi and the lists of constraints and priorities prepared running se.plan can be accomplished by calling a single function `scripts.wlc()` This function returns more information than we need to view the output so we will only take the WLC output which is the first item from the tuple.

In [ ]:
# wlc returns wlc_out, benefit_list, constraint_list, cost_list
wlc = scripts.wlc(layer_list, constraints, priorities, aoi)
output = wlc[0]

#### Optional
Optionally we can use the geemap library to view the output of or run before continuing on with generating statistics and exporting.


In [ ]:
Map = geemap.Map()
Map.centerObject(aoi,5)
Map

In [ ]:
Map.addLayer(output,{'min':0,'max':5,'palette':['000000','aff0ff','72b1ff','4564ff','592fff','8806ff']},'wlc output')


### Step 5 - get dashboard stats

In the se.plan GUI when we generate the dashboard we have to use a `.getInfo()` call to return the data to the client side. While this works it is common to run into 'computation timed out' errors. By keeping the on Googles servers we can safely export the statistics to a CSV. 

We will get the statistics for the entire region as well as map over our feature collection. 

In [ ]:
dashboard_data = scripts.get_summary_statistics(wlc, 'mekong-test', aoi, layer_list, False)
# print(scripts.dashboard_data_to_fc(dashboard_data).getInfo())

Preparing the sub aois takes a little bit more work. We will use a custom function `get_sub_aois_dashboard_data` to extract the information we want. It expects a feature collection the name of an identifying column and the unique values of which you want to run the statistics on. In this case we want to use all of them so we can use `aggregate_array` on the `ADM0_NAME` property.  

To make exporting simpler, we'll add the dashboard_data results the sub aois list as a tuple in the same format.

In [ ]:
def get_sub_aois_dashboard_data(sub_aois:ee.FeatureCollection, ids:list, ids_col:str, wlc:ee.Image, layer_list:list):
    def get_sub_stats(sub_id):
        geom = sub_aois.filter(ee.Filter.eq(ids_col,sub_id)).first().geometry()
        return (sub_id, scripts.get_summary_statistics(wlc, sub_id, geom, layer_list, False))
    return [get_sub_stats(sub_id) for sub_id in ids]
    

In [ ]:
# print(sub_aois.first().toDictionary().getInfo())
sub_id_col = 'ADM0_NAME'
sub_ids = sub_aois.aggregate_array(sub_id_col).getInfo()
# dashboard_data_sub_aois = sub_aois.toList(10).map(lambda x: scripts.get_summary_statistics(wlc, 'ok', ee.Feature(x).geometry(), layer_list, False))
# print(dashboard_data_sub_aois.size().getInfo())
dashboard_data_sub_aois = get_sub_aois_dashboard_data(sub_aois, sub_ids, sub_id_col, wlc, layer_list)
dashboard_data_sub_aois.append( ('mekong-test',dashboard_data) )
dashboard_data_sub_aois

### Step 6 - exporting 


In [ ]:
for name, eedict in dashboard_data_sub_aois:
    name = name.replace(" ","_").replace("'","")
    fc = scripts.dashboard_data_to_fc(eedict)
    ee.batch.Export.table.toDrive(
        collection=fc,
        description=name,
        folder='seplan_batch',
        fileFormat='csv',
        selectors='category,name,type,value'
    ).start()

In [ ]:
ee.batch.Export.image.toAsset(
    image = output,
    description = 'mekong-seplan-test',
    assetId = 'projects/john-ee-282116/assets/mekong-seplan-test',
    region = aoi.geometry(),
    scale = 30,
    maxPixels = 1e13
).start()

def get_stats_ee(wlc_outputs, layer_model, areas, name)



what the cli should look like,

somename recipe main_aoi sumarry_aois -n nameOfRun -summary-name-col summary_aois_name_col

ategory,